# Домашнее задание №2.
*Екатерина Волошина, БКЛ-182*
## Проверка на PEP-8

In [58]:
%load_ext pycodestyle_magic
%flake8_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


## Задание 1. Книга
В директории лежит "Красное и черное" Стендаля, откроем его в питоне и сохраним в переменную **text**.

In [59]:
with open('redblack.txt_Ascii.txt', 'r', encoding='utf-8') as file:
    text = file.read()

## Задание 2. MyStem
Для начала импортируем модуль json для дальнейшей работы с ним.

In [60]:
import json

Теперь импортируем MyStem из модуля pymystem3.

In [61]:
from pymystem3 import Mystem
m = Mystem()

Чистим текст от знаков препинания.

In [62]:
from string import punctuation
for i in punctuation:
    text = text.replace(i, '')
text = text.replace('\n', ' ')

Проводим разбор с помощью analyze и замеряем время.

In [63]:
%%time
ana = m.analyze(text)

Wall time: 17.9 s


Записываем результат анализа в файл json.

In [64]:
with open('mystem.json', 'w', encoding='utf-8') as file:
    json.dump(ana, file, ensure_ascii=False)

## Задание 3. PyMorphy
Импортируем из nltk.tokenize word_tokenize и обрабатываем с помощью него текст.

In [65]:
from nltk.tokenize import word_tokenize
words = word_tokenize(text)

Импортируем MorphAnalyzer из pymorphy2. 

In [66]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Создаем пустой список, куда будем добавлять результат анализа.

In [67]:
words_parse = []

Парсим текст, результат записываем в файл.

In [68]:
%%time
for word in words:
    words_parse.append(morph.parse(word))

Wall time: 1min 14s


Создаем пустой список, в который добавляем словари вида {токен : лемма, часть речи}.

In [69]:
dict_words = []
for word in words_parse:
    first = word[0]
    dict_w = {}
    dict_w[first.word] = first.normal_form, first.tag.POS
    dict_words.append(dict_w)

Записываем результат результат анализа в файл json.

In [70]:
with open('dict.json', 'w', encoding='utf-8') as file:
    json.dump(dict_words, file, ensure_ascii=False)

Создаем пустой список для задания со звездочкой. Записываем туда весь разбор в список словарей. В словарь записываем как ключ токен, как значение - список, содержащий весь разбор .tag

In [71]:
dict_words_2 = []
for word in words_parse:
    first = word[0]
    dict_2 = {}
    dict_2[first.word] = str(first.tag).split(',')
    dict_words_2.append(dict_2)

Записываем результат в файл json.

In [72]:
with open('dict_2.json', 'w', encoding='utf-8') as file:
    json.dump(dict_words_2, file, ensure_ascii=False)

## Задание 4. Части речи и топ глаголов и наречий.
Импортируем Counter для дальнейшего составления частотных словарей.

In [73]:
from collections import Counter

Создаем пустой список, куда записываем часть речи каждого токена, затем создаем частотный словарь на основе этого списка

In [74]:
POS_list = []
for i in dict_words:
    for value in i.values():
        POS_list.append(value[1])
counter_POS = Counter(POS_list)

### Ответ:
Части речи и их отношение к общему числу слов.

In [75]:
for key, value in counter_POS.items():
    print(key, ' - ', value/len(POS_list)*100)

NOUN  -  23.0149152869624
ADJF  -  12.433026017280495
CONJ  -  9.104841434570643
None  -  1.0383984167591833
PRCL  -  6.5477627069556394
VERB  -  12.226070376985085
ADVB  -  6.090408842979196
ADJS  -  1.8101076410677222
INFN  -  3.403002365207318
PREP  -  9.852415890331612
NPRO  -  10.847975092918858
INTJ  -  0.16713327219191967
PRTS  -  0.44407974127528116
NUMR  -  0.6757735193319496
COMP  -  0.33305980595646084
PRTF  -  0.7536081478978617
PRED  -  0.4223584495824685
GRND  -  0.8350629917459091


Создаем пустые списки для глаголов и наречий.

In [76]:
verb_list = []
adverb_list = []

Добавляем в списки леммы всех глаголов и наречий соответственно. 

In [77]:
for i in dict_words:
    for value in i.values():
        if value[1] == 'VERB':
            verb_list.append(value[0])
        elif value[1] == 'ADVB':
            adverb_list.append(value[0])

Создаем на основе списков частотне словари, упорядоченные по частотности.

In [78]:
verb_top = Counter(verb_list).most_common()
adverb_top = Counter(adverb_list).most_common()

### Ответ: 
Топ-20 глаголов:

In [79]:
for i in range(20):
    print(verb_top[i][0], ' - ', verb_top[i][1])

быть  -  2257
мочь  -  736
сказать  -  506
говорить  -  320
стать  -  230
подумать  -  208
знать  -  183
видеть  -  176
любить  -  146
думать  -  138
хотеть  -  122
отвечать  -  116
увидеть  -  109
заметить  -  100
чувствовать  -  95
дать  -  94
сделать  -  93
показаться  -  88
стоить  -  82
идти  -  81


Топ-20 наречий:

In [80]:
for i in range(20):
    print(adverb_top[i][0], ' - ', verb_top[i][1])

уже  -  2257
теперь  -  736
уж  -  506
очень  -  320
сейчас  -  230
никогда  -  208
несколько  -  183
совсем  -  176
наконец  -  146
более  -  138
тут  -  122
здесь  -  116
тогда  -  109
вдруг  -  100
всего  -  95
весьма  -  94
совершенно  -  93
там  -  88
чуть  -  82
столь  -  81


## Задание 5. Топы биграмм и триграмм.

Создаем пустой список для лемм, добавляем туда леммы всех словоформ.

In [81]:
lemmas = []
for i in words_parse:
    first = i[0]
    lemmas.append(first.normal_form)

Импортируем из nltk.util bigrams для составления биграмм.
Создаем список биграмм, на его основе создаем частотный словарь, который упорядочиваем по частотности.

In [82]:
from nltk.util import bigrams
bigrams = list(bigrams(lemmas))
top_bi = Counter(bigrams).most_common()

Аналогично создаем частотный словарь триграмм.

In [83]:
from nltk.util import trigrams
trigrams = list(trigrams(lemmas))
top_tri = Counter(trigrams).most_common()

### Ответ:
Топ-25 биграмм

In [84]:
for i in range(25):
    print(i+1, ' '.join(top_bi[i][0]), ' - ', top_bi[i][1])

1 де реналя  -  623
2 что он  -  352
3 де ламоля  -  332
4 это быть  -  237
5 гжа де  -  234
6 он не  -  214
7 тот что  -  210
8 не мочь  -  209
9 он быть  -  197
10 у он  -  195
11 который он  -  190
12 и он  -  188
13 что я  -  183
14 не быть  -  182
15 что она  -  179
16 гн де  -  171
17 если бы  -  162
18 сказать он  -  161
19 с он  -  157
20 в этот  -  154
21 он и  -  154
22 я не  -  154
23 он в  -  152
24 и не  -  148
25 в тот  -  143


Топ-25 триграмм

In [85]:
for i in range(25):
    print(i+1, ' '.join(top_tri[i][0]), ' - ', top_tri[i][1])

1 гжа де реналя  -  207
2 гн де реналя  -  101
3 мль де ламоля  -  100
4 госпожа де реналя  -  71
5 гжи де реналя  -  69
6 чуть ли не  -  64
7 о тот что  -  60
8 в конец конец  -  53
9 после тот как  -  48
10 гна де реналя  -  47
11 мадемуазель де ламоля  -  43
12 в тот что  -  41
13 на другой день  -  40
14 тот что он  -  39
15 гже де реналя  -  38
16 говорить он себя  -  38
17 господин де реналя  -  37
18 по крайний мера  -  37
19 гн де ламоля  -  36
20 гжу де реналя  -  34
21 для тот чтобы  -  32
22 до сей пора  -  30
23 в этот минута  -  30
24 де реналя и  -  29
25 в тот же  -  29
